# Création d'un RAG

## Récolte des données et préparations des chunks:

In [8]:
import pandas as pd 
import re
from bs4 import BeautifulSoup
import requests


ai_act_url = "https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=OJ:L_202401689"
soup = BeautifulSoup(requests.get(ai_act_url).content, "html.parser")
text = []
context = []
url = []
url_sections = []
url_chapitres = []


&emsp;La préparation des données et métadonné ce fait ici à l'aide de **BeautifulSoup** et non pas les [**document loaders de LangChain**]() car il nous faut plus de control sur la façon dont on séctionne l'AI Act. 

&emsp; On cherche à le séctionner en fonction de ses *Considerants*, *Chapitres*, *Sections* et *Articles*.

Chacun de ces éléments seront accompagner de métadonnées tel qu'un liens à la séction, le nom du chapitre et de l'article.

In [9]:
# Preambule Parser
considerants = soup.find_all("div", {"class":"eli-subdivision", "id": re.compile(r'rct_\d+')})
text += [considerant.text.strip() for considerant in considerants]
context += [f"Considérant {n+1}" for n in range(len(considerants))]
url += [ai_act_url+f"#rct_{n+1}" for n in range(len(considerants))]
url_sections += [None for _ in range(len(considerants))]
url_chapitres += [ai_act_url+"#pbl_1" for _ in range(len(considerants))]

In [10]:
#Chapitre Parser
chapitres = soup.find("div", {"id": "enc_1"}).find_all("div", {"id": re.compile(r'cpt_[XVILC]+')}, recursive=False)
articles = []

def get_article_and_article_str(soup_element: BeautifulSoup):
    _articles = soup_element.find_all("div", {"class":"eli-subdivision", "id": re.compile(r'art_\d+')})
    article_strs = [
        f"Art.{article['id'].strip('art_')}-"+article.find("div", {"class": "eli-title"}).text for
        article in _articles
    ]
    return _articles, article_strs

for idx_ch, chapitre in enumerate(chapitres):
    chapitre_url = ai_act_url+f"#{chapitre['id']}"
    chapitre_str = f"Ch.{idx_ch+1}-"+chapitre.find("p", {"class": "oj-ti-section-2"}).text.strip("\n")+" > "
    chapitre_articles =  chapitre.find_all("div", {"class":"eli-subdivision", "id": re.compile(r'art_\d+')})
    url_chapitres += [chapitre_url for _ in range(len(chapitre_articles))]
    text += [article.text.strip() for article in chapitre_articles]
    
    section_str = ""
    sections = chapitre.find_all("div", {"id":re.compile(r'cpt_[XVILC]+.sct_\d+')}, recursive=False)
    
    if sections:
        for idx_sct, section in enumerate(sections):
            _articles, article_strs = get_article_and_article_str(section)
            articles += _articles
            url_sections += [ai_act_url+f"#{section['id']}" for _ in range(len(_articles))]
            
            section_str = f"Sct.{idx_sct+1}-"+section.find("p", {"class": "oj-ti-section-2"}).text.strip("\n")+" > "
            context += [chapitre_str+section_str+article_str for article_str in article_strs]
    else:
        _articles, article_strs = get_article_and_article_str(chapitre)
        articles += _articles
        context += [chapitre_str+article_str for article_str in article_strs]
        url_sections += [None for _ in range(len(_articles))]
    
url += [ai_act_url+f"#art_{n}" for n in range(1,len(articles)+1)]

In [ ]:
annexes = soup.find_all("div", {"class": "eli-container", "id": re.compile(r'anx_[XVILC]+')})
titres_annexes = [annexe.find("p", {"class": "oj-doc-ti", "id":None}).get_text() for annexe in annexes]

text += [annexe.text.strip() for annexe in annexes]
context += titres_annexes
url += [ai_act_url+f"#{annexe['id']}" for annexe in annexes]
url_sections += [None for _ in range(len(annexes))]
url_chapitres += [ai_act_url+"##anx_I" for _ in range(len(annexes))]

texts = [f"{context[idx]} \n {text[idx]}" for idx in range(len(text))]
metadatas = [ 
    {
        "titre":context[idx],
        "url":url[idx],
        "url_chapitre": url_chapitres[idx],
        "url_section": url_sections[idx]
    }
    for idx in range(len(texts))
]

In [23]:
metadatas[270]

{'section_title': 'Ch.9-SURVEILLANCE APRÈS COMMERCIALISATION, PARTAGE D’INFORMATIONS ET SURVEILLANCE DU MARCHÉ > Sct.5-Surveillance, enquêtes, contrôle de l’application et contrôle en ce qui concerne les fournisseurs de modèles d’IA à\xa0usage général > Art.91-\nPouvoir de demander de la documentation et des informations\n',
 'url': 'https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=OJ:L_202401689#art_91',
 'url_chapitre': 'https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=OJ:L_202401689#cpt_IX',
 'url_section': 'https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=OJ:L_202401689#cpt_IX.sct_5'}

## Préparation des Documents et des chunks avec [RecursiveCharaterTextSplitter](https://python.langchain.com/docs/how_to/recursive_text_splitter/). 

&emsp;La séléction de la taille des chunks et leurs overlap peut avoir un impact significatif sur la performance de la recherche vectorielle par la suite et donc de la pérformance du RAG par derrieres. 

Ce [guide](https://www.machinelearningplus.com/gen-ai/optimizing-rag-chunk-size-your-definitive-guide-to-better-retrieval-accuracy/#:~:text=Optimal%20chunk%20size%20for%20RAG%20systems%20typically%20ranges,tokens%29%20provide%20better%20context%20for%20complex%20reasoning%20tasks.) donne une idée de structure d'évaluation pour détèrminer les bons paramétres. 

A titre d'example, on vas utiliser des chunks de 256 characters avec un overlap de 10%. Une petite taille de chunk fera de notre RAG un outil paré pour répondre à des questions factuelles.   

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 256, chunk_overlap = int(256*0.1), length_function=len
)

text_chunks: list[Document] = text_splitter.create_documents(texts=texts, metadatas=metadatas)

## Création d'une base vectorielle. 

### En local
On peut créer et sauvgarder nos base vectoriel localement. FAISS est un exemple de base vectorielle que l'on peu créer avec LangChain et qui a l'avantage de nous éviter de créer un Index dans Azure AI Search.

In [2]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
import os
load_dotenv()

# Définition du model utilisé pour véctoriser nos données. On utilise les clefs et endpoints du modéle de vectorisation déployé dans notre AI hub. 
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_API_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_MODEL"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    retry_max_seconds=60,
    retry_min_seconds=10,
    max_retries=5,
    show_progress_bar=True,
    chunk_size=512,
    embedding_ctx_length=1000,
    check_embedding_ctx_length=True,
    skip_empty=True
)

# Définition de l'index dans le quel nous allons stoquer notre base vectorielle.
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

  0%|          | 0/1 [00:00<?, ?it/s]

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [ ]:

vector_store = AzureSearch(
    azure_search_endpoint=os.environ["SEARCH_ENDPOINT"],
    azure_search_key=os.environ["SEARCH_KEY"],
    index_name=os.environ["SEARCH_INDEX"],
    embedding_function=embeddings.embed_query,
    additional_search_client_options={"retry_total": 4},
)
